In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from stable_audio_tools.models.autoencoders import AudioConditionalVariationalAutoEncoder
from stable_audio_tools.models import create_model_from_config
from stable_audio_tools.models.utils import copy_state_dict
from stable_audio_tools.data.utils import PadCrop_Normalized_T, Stereo, Mono

import torch
import torch.nn.functional as F
import torchaudio
from torchaudio import transforms as T
from pathlib import Path
from IPython.display import Audio

### Load Model

In [ ]:
loaded = torch.load("/home/sake/userdata/sake/stable-audio-cvae/oneshot-drums-conditional-vae/fky8t0jw/checkpoints/epoch=80-step=50000.ckpt")

/tmp/ipykernel_1073459/3718643965.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded = torch.load("/home/sake/userdata/sake/stable-audio-cvae/oneshot-drums-condition

In [4]:
loaded.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'MixedPrecisionPlugin', 'model_config'])

In [5]:
state_dict = {}
for key in loaded['state_dict']:
  if "autoencoder" in key:
    keyy = key.replace("autoencoder.", "")
    state_dict[keyy] = loaded['state_dict'][key]

In [6]:
model = create_model_from_config(loaded['model_config'])
copy_state_dict(model, state_dict)
model = model.eval()
model = model.to("cuda")

/home/sake/.local/share/virtualenvs/stable-audio-cvae-diFMr3O8/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


### Load Data

In [7]:
pad_crop = PadCrop_Normalized_T(n_samples=65536, sample_rate=44100)
to_stereeo = Stereo() 

In [8]:
# audio_fn = Path("/home/sake/userdata/sake/oneshot_data/percussion/Medicated OHat 6.wav")
# audio_fn = Path("/home/sake/userdata/sake/oneshot_data/snare/07_Snare_05_SP.wav")
audio_fn = Path("/home/sake/userdata/sake/oneshot_data/kick/001_50S1.wav")
wav, sr = torchaudio.load(audio_fn)
if sr != 44100:
  resample_tf = T.Resample(sr, 44100)
  wav = resample_tf(wav)
wav, t_start, t_end, seconds_start, seconds_total, padding_mask = pad_crop(wav)
wav = wav.clamp(-1, 1)
wav = to_stereeo(wav)

In [9]:
wav.shape

torch.Size([2, 65536])

- 'kick': 0, 
- 'snare': 1, 
- 'hat': 2,
- 'clap': 3,
- 'percussion': 4
- None: 5 ?

In [10]:
og_labels = torch.tensor([0])
tgt_labels = torch.tensor([1])
none_labels = torch.tensor([5])
wav.shape, og_labels, tgt_labels

(torch.Size([2, 65536]), tensor([0]), tensor([1]))

### Reconstruct

In [11]:
with torch.no_grad():
  latents = model.encode(wav.unsqueeze(0).to("cuda"), condition=og_labels.unsqueeze(0).to("cuda"))
  twist_rec = model.decode(latents, condition=tgt_labels.unsqueeze(0).to("cuda"))

In [12]:
with torch.no_grad():
  latents = model.encode(wav.unsqueeze(0).to("cuda"), condition=none_labels.unsqueeze(0).to("cuda"))
  none_tgt_rec = model.decode(latents, condition=tgt_labels.unsqueeze(0).to("cuda"))

In [13]:
with torch.no_grad():
  latents = model.encode(wav.unsqueeze(0).to("cuda"), condition=tgt_labels.unsqueeze(0).to("cuda"))
  tgt_rec = model.decode(latents, condition=tgt_labels.unsqueeze(0).to("cuda"))

In [14]:
og_rec = model(wav.unsqueeze(0).to("cuda"), og_labels.unsqueeze(0).to("cuda"))

In [15]:
none_rec = model(wav.unsqueeze(0).to("cuda"), none_labels.unsqueeze(0).to("cuda"))

In [16]:
Audio(wav[0].numpy(), rate=44100)

In [17]:
Audio(og_rec[0].cpu().detach().numpy(), rate=44100)

In [18]:
Audio(twist_rec[0].cpu().detach().numpy(), rate=44100)

In [19]:
Audio(tgt_rec[0].cpu().detach().numpy(), rate=44100)

In [20]:
Audio(none_tgt_rec[0].cpu().detach().numpy(), rate=44100)

In [21]:
Audio(none_rec[0].cpu().detach().numpy(), rate=44100)

In [25]:
latents.shape

torch.Size([1, 64, 64])

### Sampling

- 'kick': 0, 
- 'snare': 1, 
- 'hat': 2,
- 'clap': 3,
- 'percussion': 4
- None: 5 ?

In [37]:
tgt_labels = torch.tensor([2])
none_labels = torch.tensor([5])

In [50]:
# Sample random latents from standard normal distribution
sampled_latents = 0.1 * torch.randn(1, 64, 64, device="cuda")

# 실제 latent 분포로 샘플해야함

# Generate audio from sampled latents using different conditions
none_gen = model.decode(sampled_latents, none_labels.unsqueeze(0).to("cuda"))
tgt_gen = model.decode(sampled_latents, tgt_labels.unsqueeze(0).to("cuda"))

In [51]:
Audio(none_gen[0].cpu().detach().numpy(), rate=44100)

In [52]:
Audio(tgt_gen[0].cpu().detach().numpy(), rate=44100)